<a href="https://colab.research.google.com/github/Prithvi-0731/paligemma-vqav2-replication/blob/main/replication_of_paligemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q "transformers>=4.43.0" "datasets>=2.18.0" "accelerate" "bitsandbytes" "huggingface_hub"

import torch
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from datasets import load_dataset
from huggingface_hub import login

print("Libraries installed and imported!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 41.5 MB/s eta 0:00:00
Libraries installed and imported!


In [ ]:
# --- Cell 2: Login to Hugging Face (for gated paligemma models) ---
from huggingface_hub import login

login()  # Paste your HF token when prompted

In [ ]:
# --- Cell 3: Load a local validation subset from lmms-lab/VQAv2 ---

from datasets import load_dataset

# Take first 20,000 validation examples – big enough for solid stats, small enough for Colab
vqa_val = load_dataset("lmms-lab/VQAv2", split="validation[:20000]")

print("Dataset size:", len(vqa_val))
sample = vqa_val[0]
print(sample.keys())
sample

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/962 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/68 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/36 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/143 [00:00<?, ?it/s]

...

Generating validation split:   0%|          | 0/214354 [00:00<?, ? examples/s]

Generating testdev split:   0%|          | 0/107394 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/447793 [00:00<?, ? examples/s]

Dataset size: 20000
dict_keys(['question_type', 'multiple_choice_answer', 'answers', 'image_id', 'answer_type', 'question_id', 'question', 'image'])


{'question_type': 'none of the above',
 'multiple_choice_answer': 'down',
 'answers': [{'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 1},
  {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 2},
  {'answer': 'at table', 'answer_confidence': 'yes', 'answer_id': 3},
  {'answer': 'skateboard', 'answer_confidence': 'yes', 'answer_id': 4},
  {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 5},
  {'answer': 'table', 'answer_confidence': 'yes', 'answer_id': 6},
  {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 7},
  {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 8},
  {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 9},
  {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 10}],
 'image_id': 262148,
 'answer_type': 'other',
 'question_id': 262148000,
 'question': 'Where is he looking?',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x512>}

In [ ]:
# --- Cell 4: Official VQA accuracy metric ---

import re

def normalize_answer(text: str) -> str:
    if text is None:
        return ""
    text = text.lower().strip()
    # remove punctuation
    text = re.sub(r"[^\w\s]", "", text)
    # collapse whitespace
    text = re.sub(r"\s+", " ", text)
    return text

def vqa_official_score(prediction: str, answers_list):
    """
    prediction: string from model
    answers_list: list of strings (10 human answers)
    """
    if prediction is None:
        return 0.0

    pred_norm = normalize_answer(prediction)
    gt_norms = [normalize_answer(a) for a in answers_list]

    # count how many of the 10 human answers exactly match
    matches = sum(1 for a in gt_norms if a == pred_norm)

    # official VQA acc: min(#matches / 3, 1)
    return min(matches / 3.0, 1.0)

# Quick sanity check on first example
ex0 = vqa_val[0]
all_answers0 = [a["answer"] for a in ex0["answers"]]

print("Q:", ex0["question"])
print("All human answers:", all_answers0)
print("Majority (multiple_choice_answer):", ex0["multiple_choice_answer"])

for test_pred in ["down", "at table", "skateboard", "dog"]:
    score = vqa_official_score(test_pred, all_answers0)
    print(f"Pred='{test_pred}' -> score={score:.3f}")

Q: Where is he looking?
All human answers: ['down', 'down', 'at table', 'skateboard', 'down', 'table', 'down', 'down', 'down', 'down']
Majority (multiple_choice_answer): down
Pred='down' -> score=1.000
Pred='at table' -> score=0.333
Pred='skateboard' -> score=0.333
Pred='dog' -> score=0.000


In [ ]:
# --- Cell 5: Load PaliGemma-3B fine-tuned on VQAv2 (224px) ---

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model_id_224 = "google/paligemma-3b-ft-vqav2-224"  # finetuned VQAv2 checkpoint (224px)

dtype = torch.bfloat16 if device == "cuda" else torch.float32

model_224 = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id_224,
    torch_dtype=dtype,
    device_map="auto" if device == "cuda" else None,
).eval()

processor_224 = AutoProcessor.from_pretrained(model_id_224)

print("Model and processor loaded.")

`torch_dtype` is deprecated! Use `dtype` instead!


Using device: cuda


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

Model and processor loaded.


In [ ]:
# --- Cell 6: Prediction + evaluation for paligemma-3b-ft-vqav2-224 ---

from tqdm.auto import tqdm
import numpy as np

def predict_vqa_224(example, max_new_tokens: int = 8) -> str:
    """
    Run paligemma-3b-ft-vqav2-224 on a single VQAv2 example.
    Returns the raw decoded answer string.
    """
    img = example["image"]        # already a PIL image
    question = example["question"]

    # For faithful replication, we keep the prompt simple:
    prompt = "<image> " + question

    inputs = processor_224(
        text=prompt,
        images=img,
        return_tensors="pt",
    ).to(device)

    input_len = inputs["input_ids"].shape[-1]

    with torch.no_grad():
        output_ids = model_224.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,      # greedy decoding (paper-style)
        )

    gen_ids = output_ids[0, input_len:]
    pred = processor_224.decode(gen_ids, skip_special_tokens=True)

    # keep only the first line / segment; strip whitespace
    pred = pred.strip().split("\n")[0]
    return pred


def evaluate_model_224(dataset, max_examples: int = 2000):
    """
    Evaluate the model on the first `max_examples` using the *official* VQA metric.
    """
    n = min(len(dataset), max_examples)
    total_score = 0.0
    used = 0

    for i in tqdm(range(n)):
        ex = dataset[i]
        answers_list = [a["answer"] for a in ex["answers"]]

        try:
            pred = predict_vqa_224(ex)
        except Exception as e:
            # skip any weird failures but don't crash
            continue

        score = vqa_official_score(pred, answers_list)
        total_score += score
        used += 1

    acc = (total_score / used) * 100.0 if used > 0 else 0.0
    print(f"Evaluated {used} examples out of first {n}")
    print(f"Official VQA Accuracy (224, val subset): {acc:.2f}%")
    return acc

# First, sanity check on a small subset (e.g., 200 examples)
acc_224_200 = evaluate_model_224(vqa_val, max_examples=200)

  0%|          | 0/200 [00:00<?, ?it/s]

Evaluated 200 examples out of first 200
Official VQA Accuracy (224, val subset): 92.67%


In [ ]:
# --- Cell 7: Full validation subset evaluation (20k examples) ---

acc_224_20000 = evaluate_model_224(vqa_val, max_examples=20000)
print("Final 224px model accuracy on 20k val examples (official VQA metric): "
      f"{acc_224_20000:.2f}%")

  0%|          | 0/20000 [00:00<?, ?it/s]

Evaluated 19955 examples out of first 20000
Official VQA Accuracy (224, val subset): 92.15%
Final 224px model accuracy on 20k val examples (official VQA metric): 92.15%


In [ ]:
final_acc_224 = acc_224_20000
print("Final 224px accuracy (20k val, official VQA metric):", final_acc_224)

Final 224px accuracy (20k val, official VQA metric): 92.15234277123538


In [ ]:
with open("accuracy_224_val20k.txt", "w") as f:
    f.write(f"{final_acc_224:.4f}\n")

In [12]:
!rm -rf /root/.cache/huggingface/hub/models--google--paligemma-3b-ft-vqav2-224
!rm -rf /root/.cache/huggingface/hub/google__paligemma-3b-ft-vqav2-224

In [14]:
!rm -rf /root/.cache/huggingface/hub/*google--paligemma*
!rm -rf /root/.cache/huggingface/hub/*google__paligemma*
!rm -rf /root/.cache/huggingface/hub/models--google*

In [17]:
!rm -rf /root/.cache/huggingface/hub/models*
!rm -rf /root/.cache/huggingface/hub/checkpoints*
!rm -rf /root/.cache/huggingface/hub/*tokenizer*
!rm -rf /root/.cache/huggingface/hub/*config*

In [19]:
!rm -rf /root/.cache/huggingface/hub/models--google*
!rm -rf /root/.cache/huggingface/hub/model*
!rm -rf /root/.cache/huggingface/hub/checkpoints*

In [22]:
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model_id_448 = "google/paligemma-3b-ft-vqav2-448"  # 448px fine-tuned VQAv2 model
dtype = torch.bfloat16 if device == "cuda" else torch.float32

model_448 = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id_448,
    torch_dtype=dtype,
    device_map="auto" if device == "cuda" else None,
).eval()

processor_448 = AutoProcessor.from_pretrained(model_id_448)

print("448px model and processor loaded.")

Using device: cuda


config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

448px model and processor loaded.


In [23]:
from tqdm.auto import tqdm

def predict_vqa_448(example, max_new_tokens: int = 8) -> str:
    """
    Run paligemma-3b-ft-vqav2-448 on a single VQAv2 example.
    Returns the raw decoded answer string.
    """
    img = example["image"]
    question = example["question"]

    prompt = "<image> " + question  # same style as 224 for fair comparison

    inputs = processor_448(
        text=prompt,
        images=img,
        return_tensors="pt",
    ).to(device)

    input_len = inputs["input_ids"].shape[-1]

    with torch.no_grad():
        output_ids = model_448.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,    # greedy, like before
        )

    gen_ids = output_ids[0, input_len:]
    pred = processor_448.decode(gen_ids, skip_special_tokens=True)
    pred = pred.strip().split("\n")[0]
    return pred


def evaluate_model_448(dataset, max_examples: int = 2000):
    """
    Evaluate the 448px model on the first `max_examples`
    using the official VQA metric (same as 224).
    """
    n = min(len(dataset), max_examples)
    total_score = 0.0
    used = 0

    for i in tqdm(range(n)):
        ex = dataset[i]
        answers_list = [a["answer"] for a in ex["answers"]]

        try:
            pred = predict_vqa_448(ex)
        except Exception as e:
            continue

        score = vqa_official_score(pred, answers_list)
        total_score += score
        used += 1

    acc = (total_score / used) * 100.0 if used > 0 else 0.0
    print(f"Evaluated {used} examples out of first {n}")
    print(f"Official VQA Accuracy (448, val subset): {acc:.2f}%")
    return acc

In [25]:
acc_224_2000 = evaluate_model_224(vqa_val, max_examples=2000)
print("224px model accuracy on 2k val examples:", acc_224_2000)

  0%|          | 0/2000 [00:00<?, ?it/s]

Evaluated 2000 examples out of first 2000
Official VQA Accuracy (224, val subset): 92.82%
224px model accuracy on 2k val examples: 92.81666666666666


In [26]:
acc_448_2000 = evaluate_model_448(vqa_val, max_examples=2000)
print("448px model accuracy on 2k val examples:", acc_448_2000)

  0%|          | 0/2000 [00:00<?, ?it/s]

Evaluated 2000 examples out of first 2000
Official VQA Accuracy (448, val subset): 93.27%
448px model accuracy on 2k val examples: 93.26666666666671


In [27]:
import re

# Map number words to digits for more robust matching
NUMBER_MAP = {
    "zero": "0", "one": "1", "two": "2", "three": "3",
    "four": "4", "five": "5", "six": "6", "seven": "7",
    "eight": "8", "nine": "9", "ten": "10"
}

def normalize_answer_custom(s: str) -> str:
    """
    Your custom normalization:
    - lowercase
    - strip spaces
    - remove punctuation
    - map number words to digits
    """
    if s is None:
        return ""
    s = str(s)
    s = s.strip().lower()
    # remove punctuation
    s = re.sub(r"[^\w\s]", "", s)
    # collapse multiple spaces
    s = re.sub(r"\s+", " ", s)
    # map number words -> digits
    tokens = s.split()
    mapped = [NUMBER_MAP.get(tok, tok) for tok in tokens]
    return " ".join(mapped)

In [28]:
from tqdm.auto import tqdm

def predict_vqa_224_variant(
    example,
    max_new_tokens: int = 8,
    use_beam: bool = False,
    prompt_style: str = "plain"  # "plain" or "qa"
) -> str:
    """
    Variant predictor for the 224px model with:
      - greedy vs beam search
      - different prompt styles
      - configurable max_new_tokens
    """
    img = example["image"]
    question = example["question"]

    # Prompt styles
    if prompt_style == "qa":
        # Your designed QA-style prompt
        prompt = f"<image> Question: {question} Answer:"
    else:
        # Baseline prompt, same as before
        prompt = "<image> " + question

    inputs = processor_224(
        text=prompt,
        images=img,
        return_tensors="pt",
    ).to(device)

    # Base generation kwargs
    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        do_sample=False,   # deterministic
    )

    # Optional beam search
    if use_beam:
        gen_kwargs.update(
            num_beams=3,
            early_stopping=True
        )

    with torch.no_grad():
        output_ids = model_224.generate(**inputs, **gen_kwargs)

    # Slice off the prompt tokens
    input_len = inputs["input_ids"].shape[-1]
    gen_ids = output_ids[0, input_len:]
    pred = processor_224.decode(gen_ids, skip_special_tokens=True)
    pred = pred.strip().split("\n")[0]
    return pred

In [29]:
def evaluate_model_224_variant(
    dataset,
    max_examples: int = 500,
    use_beam: bool = False,
    prompt_style: str = "plain",  # "plain" or "qa"
    max_new_tokens: int = 8,
    use_norm: bool = False,
    desc: str = ""
):
    """
    Evaluate the 224px model with different decoding / prompt / normalization settings.
    Uses your existing vqa_official_score for scoring.
    """
    n = min(len(dataset), max_examples)
    total_score = 0.0
    used = 0

    for i in tqdm(range(n), desc=desc):
        ex = dataset[i]
        answers_list = [a["answer"] for a in ex["answers"]]

        try:
            pred = predict_vqa_224_variant(
                ex,
                max_new_tokens=max_new_tokens,
                use_beam=use_beam,
                prompt_style=prompt_style,
            )
        except Exception:
            # skip problematic examples
            continue

        if use_norm:
            pred_for_score = normalize_answer_custom(pred)
            answers_for_score = [normalize_answer_custom(a) for a in answers_list]
        else:
            pred_for_score = pred
            answers_for_score = answers_list

        score = vqa_official_score(pred_for_score, answers_for_score)
        total_score += score
        used += 1

    acc = (total_score / used) * 100.0 if used > 0 else 0.0
    print(f"Evaluated {used} examples out of first {n}")
    print(
        f"Variant (beam={use_beam}, prompt={prompt_style}, "
        f"max_new_tokens={max_new_tokens}, norm={use_norm}) "
        f"Accuracy: {acc:.2f}%"
    )
    return acc

In [30]:
N_IMPROVE = 500

print("Baseline (greedy, plain prompt, no extra norm, max_new_tokens=8)")
acc_base_500 = evaluate_model_224_variant(
    vqa_val,
    max_examples=N_IMPROVE,
    use_beam=False,
    prompt_style="plain",
    max_new_tokens=8,
    use_norm=False,
    desc="Baseline"
)

print("\nBeam search (beam=3) – your improvement A")
acc_beam_500 = evaluate_model_224_variant(
    vqa_val,
    max_examples=N_IMPROVE,
    use_beam=True,
    prompt_style="plain",
    max_new_tokens=8,
    use_norm=False,
    desc="Beam search"
)

print("\nPrompt engineering ('Question: ... Answer:') – your improvement B")
acc_prompt_500 = evaluate_model_224_variant(
    vqa_val,
    max_examples=N_IMPROVE,
    use_beam=False,
    prompt_style="qa",
    max_new_tokens=8,
    use_norm=False,
    desc="Prompt-QA"
)

print("\nBeam + prompt + extra normalization – your combo improvement C")
acc_combo_500 = evaluate_model_224_variant(
    vqa_val,
    max_examples=N_IMPROVE,
    use_beam=True,
    prompt_style="qa",
    max_new_tokens=8,
    use_norm=True,
    desc="Beam+Prompt+Norm"
)

acc_base_500, acc_beam_500, acc_prompt_500, acc_combo_500

Baseline (greedy, plain prompt, no extra norm, max_new_tokens=8)


Baseline:   0%|          | 0/500 [00:00<?, ?it/s]

Evaluated 500 examples out of first 500
Variant (beam=False, prompt=plain, max_new_tokens=8, norm=False) Accuracy: 93.27%

Beam search (beam=3) – your improvement A


Beam search:   0%|          | 0/500 [00:00<?, ?it/s]

Evaluated 500 examples out of first 500
Variant (beam=True, prompt=plain, max_new_tokens=8, norm=False) Accuracy: 92.73%

Prompt engineering ('Question: ... Answer:') – your improvement B


Prompt-QA:   0%|          | 0/500 [00:00<?, ?it/s]

Evaluated 500 examples out of first 500
Variant (beam=False, prompt=qa, max_new_tokens=8, norm=False) Accuracy: 93.27%

Beam + prompt + extra normalization – your combo improvement C


Beam+Prompt+Norm:   0%|          | 0/500 [00:00<?, ?it/s]

Evaluated 500 examples out of first 500
Variant (beam=True, prompt=qa, max_new_tokens=8, norm=True) Accuracy: 92.80%


(93.26666666666668, 92.73333333333336, 93.26666666666668, 92.80000000000001)

In [31]:
def predict_vqa_224_variant(
    example,
    max_new_tokens: int = 8,
    use_beam: bool = False,
    prompt_style: str = "plain"  # "plain", "qa", or "qa_short"
) -> str:
    """
    Variant predictor for the 224px model with:
      - greedy vs beam search
      - different prompt styles
      - configurable max_new_tokens
    """
    img = example["image"]
    question = example["question"]

    # Prompt styles
    if prompt_style == "qa":
        prompt = f"<image> Question: {question} Answer:"
    elif prompt_style == "qa_short":
        # Your new strict, short-answer instruction
        prompt = (
            f"<image> Question: {question} "
            f"Answer with a single short word or number:"
        )
    else:
        # Baseline prompt
        prompt = "<image> " + question

    inputs = processor_224(
        text=prompt,
        images=img,
        return_tensors="pt",
    ).to(device)

    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        do_sample=False,
    )
    if use_beam:
        gen_kwargs.update(
            num_beams=3,
            early_stopping=True
        )

    with torch.no_grad():
        output_ids = model_224.generate(**inputs, **gen_kwargs)

    input_len = inputs["input_ids"].shape[-1]
    gen_ids = output_ids[0, input_len:]
    pred = processor_224.decode(gen_ids, skip_special_tokens=True)
    pred = pred.strip().split("\n")[0]
    return pred

In [32]:
N_IMPROVE = 500  # same subset size

print("Baseline again (for reference)")
acc_base_500_repeat = evaluate_model_224_variant(
    vqa_val,
    max_examples=N_IMPROVE,
    use_beam=False,
    prompt_style="plain",
    max_new_tokens=8,
    use_norm=False,
    desc="Baseline-ref"
)

print("\nNew variant: QA-short, max_new_tokens=4, with normalization")
acc_qa_short_500 = evaluate_model_224_variant(
    vqa_val,
    max_examples=N_IMPROVE,
    use_beam=False,
    prompt_style="qa_short",
    max_new_tokens=4,
    use_norm=True,
    desc="QA-short"
)

acc_base_500_repeat, acc_qa_short_500

Baseline again (for reference)


Baseline-ref:   0%|          | 0/500 [00:00<?, ?it/s]

Evaluated 500 examples out of first 500
Variant (beam=False, prompt=plain, max_new_tokens=8, norm=False) Accuracy: 93.27%

New variant: QA-short, max_new_tokens=4, with normalization


QA-short:   0%|          | 0/500 [00:00<?, ?it/s]

Evaluated 500 examples out of first 500
Variant (beam=False, prompt=qa_short, max_new_tokens=4, norm=True) Accuracy: 82.40%


(93.26666666666668, 82.39999999999998)

In [33]:
N_ENSEMBLE = 500

print("448-only baseline on 500 examples (for ensemble comparison)")
acc_448_500 = evaluate_model_448(
    vqa_val,
    max_examples=N_ENSEMBLE
)
acc_448_500

448-only baseline on 500 examples (for ensemble comparison)


  0%|          | 0/500 [00:00<?, ?it/s]

Evaluated 500 examples out of first 500
Official VQA Accuracy (448, val subset): 93.13%


93.13333333333334

In [34]:
from collections import Counter
from tqdm.auto import tqdm

def evaluate_ensemble_224_448(
    dataset,
    max_examples: int = 500,
):
    n = min(len(dataset), max_examples)
    total_score = 0.0
    used = 0

    for i in tqdm(range(n), desc="Ensemble 224+448"):
        ex = dataset[i]
        answers_list = [a["answer"] for a in ex["answers"]]

        try:
            # 224 baseline prediction (your standard setup)
            pred_224 = predict_vqa_224_variant(
                ex,
                max_new_tokens=8,
                use_beam=False,
                prompt_style="plain",
            )

            # 448 baseline prediction (same as before)
            pred_448 = predict_vqa_448(ex)

        except Exception:
            continue

        # Normalize all answers
        norm_224 = normalize_answer_custom(pred_224)
        norm_448 = normalize_answer_custom(pred_448)
        norm_gt = [normalize_answer_custom(a) for a in answers_list]

        # Simple ensemble rule:
        # - if 224 and 448 agree, use that
        # - otherwise trust 448 (higher-capacity model)
        if norm_224 == norm_448 and norm_224 != "":
            final_pred = norm_224
        else:
            final_pred = norm_448

        score = vqa_official_score(final_pred, norm_gt)
        total_score += score
        used += 1

    acc = (total_score / used) * 100.0 if used > 0 else 0.0
    print(f"Evaluated {used} examples out of first {n}")
    print(f"Ensemble 224+448 accuracy (500 examples): {acc:.2f}%")
    return acc

# Run on the same 500-example subset
acc_ens_500 = evaluate_ensemble_224_448(vqa_val, max_examples=N_ENSEMBLE)
acc_ens_500

Ensemble 224+448:   0%|          | 0/500 [00:00<?, ?it/s]

Evaluated 500 examples out of first 500
Ensemble 224+448 accuracy (500 examples): 93.13%


93.13333333333334

In [35]:
from typing import List

def predict_vqa_224_sampled(
    example,
    n_samples: int = 3,
    temperature: float = 0.7,
    top_p: float = 0.9,
    max_new_tokens: int = 8,
) -> List[str]:
    """
    Generate n_samples answers from the 224px model using sampling
    (do_sample=True, with temperature and top_p).
    Returns a list of raw decoded answer strings.
    """
    img = example["image"]
    question = example["question"]

    prompt = "<image> " + question   # same as baseline prompt

    inputs = processor_224(
        text=prompt,
        images=img,
        return_tensors="pt",
    ).to(device)

    answers = []

    for _ in range(n_samples):
        with torch.no_grad():
            output_ids = model_224.generate(
                **inputs,
                do_sample=True,
                temperature=temperature,
                top_p=top_p,
                max_new_tokens=max_new_tokens,
            )
        input_len = inputs["input_ids"].shape[-1]
        gen_ids = output_ids[0, input_len:]
        pred = processor_224.decode(gen_ids, skip_special_tokens=True)
        pred = pred.strip().split("\n")[0]
        answers.append(pred)

    return answers

In [36]:
from collections import Counter
from tqdm.auto import tqdm

def evaluate_model_224_sampling_ensemble(
    dataset,
    max_examples: int = 500,
    n_samples: int = 3,
    temperature: float = 0.7,
    top_p: float = 0.9,
    max_new_tokens: int = 8,
):
    """
    Evaluate 224px model with sampling + majority vote over multiple generations.
    Falls back to greedy baseline when there's no clear majority.
    """
    n = min(len(dataset), max_examples)
    total_score = 0.0
    used = 0

    for i in tqdm(range(n), desc="224 sampling+vote"):
        ex = dataset[i]
        answers_list = [a["answer"] for a in ex["answers"]]
        norm_gt = [normalize_answer_custom(a) for a in answers_list]

        try:
            # 1) Get greedy baseline prediction (for fallback)
            greedy_pred = predict_vqa_224_variant(
                ex,
                max_new_tokens=max_new_tokens,
                use_beam=False,
                prompt_style="plain",
            )
            greedy_norm = normalize_answer_custom(greedy_pred)

            # 2) Get multiple sampled predictions
            sampled_preds = predict_vqa_224_sampled(
                ex,
                n_samples=n_samples,
                temperature=temperature,
                top_p=top_p,
                max_new_tokens=max_new_tokens,
            )
            sampled_norm = [normalize_answer_custom(p) for p in sampled_preds]

        except Exception:
            continue

        # Count non-empty normalized answers
        valid_answers = [a for a in sampled_norm if a != ""]
        if len(valid_answers) == 0:
            final_pred = greedy_norm
        else:
            counts = Counter(valid_answers)
            # Best candidate and its count
            best_answer, best_count = counts.most_common(1)[0]

            # If clear majority or best_answer != empty, use it, else fallback
            if best_count >= 2:
                final_pred = best_answer
            else:
                # No strong agreement -> use greedy
                final_pred = greedy_norm

        score = vqa_official_score(final_pred, norm_gt)
        total_score += score
        used += 1

    acc = (total_score / used) * 100.0 if used > 0 else 0.0
    print(f"Evaluated {used} examples out of first {n}")
    print(
        f"224 sampling+majority (n={n_samples}, T={temperature}, top_p={top_p}) "
        f"Accuracy: {acc:.2f}%"
    )
    return acc

# Run on the same 500-example subset
N_SAMPLING = 500
acc_224_sampling_500 = evaluate_model_224_sampling_ensemble(
    vqa_val,
    max_examples=N_SAMPLING,
    n_samples=3,
    temperature=0.7,
    top_p=0.9,
    max_new_tokens=8,
)
acc_224_sampling_500

224 sampling+vote:   0%|          | 0/500 [00:00<?, ?it/s]

Evaluated 500 examples out of first 500
224 sampling+majority (n=3, T=0.7, top_p=0.9) Accuracy: 92.13%


92.13333333333334

In [37]:
with open("accuracy_448_2k.txt", "w") as f:
    f.write("PaliGemma-3B VQAv2 448px Accuracy (2,000 examples): 93.27%\n")

print("Created: accuracy_448_2k.txt")

Created: accuracy_448_2k.txt


In [38]:
import pandas as pd

ablation_data = [
    ["Baseline (224, greedy)", "224", "greedy", "plain", "No", "No", 93.27],
    ["Beam search (beam=3)", "224", "beam=3", "plain", "No", "No", 92.73],
    ["QA-style prompt", "224", "greedy", "qa", "No", "No", 93.27],
    ["Strict short-answer", "224", "greedy", "qa_short", "No", "Yes", 82.40],
    ["Sampling + majority vote", "224", "greedy", "plain", "3 samples", "Yes", 92.13],
    ["High resolution (448)", "448", "greedy", "plain", "No", "No", 93.13],
    ["224+448 ensemble", "224+448", "greedy", "plain", "No", "Yes", 93.13],
]

df = pd.DataFrame(
    ablation_data,
    columns=[
        "Experiment",
        "Resolution",
        "Decoding",
        "Prompt",
        "Sampling",
        "Normalization",
        "Accuracy (%)"
    ]
)

df.to_csv("ablation_table.csv", index=False)
print("Created: ablation_table.csv")
df

Created: ablation_table.csv


,Experiment,Resolution,Decoding,Prompt,Sampling,Normalization,Accuracy (%)
0,"Baseline (224, greedy)",224,greedy,plain,No,No,93.27
1,Beam search (beam=3),224,beam=3,plain,No,No,92.73
2,QA-style prompt,224,greedy,qa,No,No,93.27
3,Strict short-answer,224,greedy,qa_short,No,Yes,82.40
4,Sampling + majority vote,224,greedy,plain,3 samples,Yes,92.13
5,High resolution (448),448,greedy,plain,No,No,93.13
6,224+448 ensemble,224+448,greedy,plain,No,Yes,93.13
